# DQN и PacMan
### Вступление

Привет! Сегодня мы вместе реализуем алгоритм DDQN, который является одним из самых простых методов deep reinforcement learning, однако при этом показывает достаточно высокие результаты на большом количестве задач. Более того, архитектура, приведенная в [статье](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf) позволяет эффективно решать целый спектр различных задач без адаптации алгоритма к их условиям.

### Требования
Для начала разберемся с тем, что мы будем использовать. Во-первых, нам понадобится Python 3.5+ и Linux (к сожалению, некоторые библиотеки могут некорректно работать с другими ОС). Во-вторых, нам потребуются следующие библиотеки для Python:
* `gym` - Общий интерфейс для многих сред RL. Рекомендуется устанавливать вместе с пакетом игр atari, т.к. это позволит в дальнейшем не исправлять зависимости вручную. Команда для установки: `pip install gym[atari]`
* `numpy` - Библиотека для работы с математикой. Команда для установки: `pip install numpy`
* `pytorch` - Фреймворк для реализации и обучения нейронных сетей. [Инструкция по установке](https://pytorch.org/get-started/locally/)
* `matplotlib` - Потребуется нам для отображения картинок и графиков. Команда для установки: `pip install matplotlib`

После установки всего необходимого, нужно импортировать все нужные нам библиотеки в скрипт:

In [1]:
import gym
import matplotlib.pyplot as plt
import copy
import random
import numpy as np
from collections import deque
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

#Определяем, можем ли использовать cuda для вычислений на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### До создания модели
Теперь создадим окружение, на котором будем тестировать алгоритм. В качестве примера будем использовать игру *PacMan*

In [ ]:
# Create environment
env = gym.make("MsPacman-v0")
frame = env.reset()

In [ ]:
#Отобразим фрейм из начала игры
plt.imshow(frame)

Естественно, мы не будем использовать исходное изображение в качестве наблюдения. Оно слишком большое и цветное. Мы сделаем его черно-белым и уменьшим до размера 84x84 с помощью масштабирования и обрезания

In [3]:
def to_grayscale(image):
    #Константы взяты из https://en.wikipedia.org/wiki/Grayscale
    return 0.2126 * image[:, :, 0] + 0.7152 * image[:, :, 1] + 0.0722 * image[:, :, 2]


def preprocesing(image):
    grayscale = to_grayscale(image)
    #Предполагаем, что изображение имеет размер 160x210
    #После такой трансформации изображение будет отмаштабировано, а нижняя часть - обрезана
    return [[grayscale[i * 160 // 84, j * 160 // 84]for j in range(84)] for i in range(84)]

In [4]:
#Посмотрим на результат
plt.imshow(preprocesing(frame), cmap="gray")

NameError: name 'frame' is not defined

Поскольку DQN представляет собой нейронную сеть, то обучаться она будет, как и любая другая нейронная сеть, при помощи батчей. Обычно батчи выбираются случайно из набора данных, однако в нашем случае его нет, а есть только среда и возможность взаимодействовать с ней. Решение этой проблемы очень простое: давайте создадим буфер ограниченного размера, в который будем складывать replay'и, полученные в результате взаимодействия со средой, при этом удаляя самые старые, если не хватает места. Этот буфер мы будем считать нашим набором данных и брать батчи прямо из него.

In [ ]:
class Memory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, element):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = element
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


memory = Memory(10000)

### Описание модели
Теперь пришло время описать архитектуру самой нейронной сети. Первые три слоя - сверточные, затем идет один линейный слой, которой выделяет признаки и в конце слой, который комбинирует эти признаки в Q-function для каждого действия в данном состоянии

In [ ]:
model = nn.Sequential(
    nn.Conv2d(4, 32, 8, 4),
    nn.ReLU(),
    nn.Conv2d(32, 64, 4, 2),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, 1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(3136, 512),
    nn.ReLU(),
    nn.Linear(512, 5)
)

target_model = copy.deepcopy(model)

In [ ]:
#Инициализация весов нейронной сети
def init_weights(layer):
    if type(layer) == nn.Linear or type(layer) == nn.Conv2d:
        nn.init.xavier_normal(layer.weight)


model.apply(init_weights)

In [ ]:
#Загружаем модель на устройство, определенное в самом начале (GPU или CPU)
model.train()
target_model.train()
model.to(device)
target_model.to(device)

Сразу зададим оптимизатор, с помощью которого будем обновлять веса модели

In [ ]:
optimizer = optim.RMSprop(model.parameters(), lr=0.00025, alpha=0.95, eps=0.01)
gamma = 0.99

Осталось описать функцию, которая будет по батчу считать функцию потерь и запускать обновление весов

In [ ]:
def fit(batch):
        state, action, reward, next_state, done = batch
        state = torch.tensor(state).to(device).float()
        next_state = torch.tensor(next_state).to(device).float()
        reward = torch.tensor(reward).to(device).float()
        action = torch.tensor(action).to(device)

        target_q = torch.zeros(reward.size()[0]).float().to(device)
        with torch.no_grad():
            #Get predicted by target model Q-function
            target_q[done] = target_model(next_state).max(1)[0].detach()
        #Estimate current Q-function
        target_q = reward + target_q * gamma

        #Current approximation
        q = model(state).gather(1, action.unsqueeze(1))

        loss = F.smooth_l1_loss(q, target_q.unsqueeze(1))

        #Clear all gradients of network
        optimizer.zero_grad()
        #Backpropagate loss
        loss.backward()
        #Clip gradient
        for param in model.parameters():
            param.grad.data.clamp_(-1, 1)
        #Update network parameters
        optimizer.step()

Также опишем $\Epsilon$-greedy политику, которая совершает случайное действие с вероятностью $\Epsilon$ в целях исследования среды.

In [ ]:
#Вероятность совершить случайное действие(exploration probability)
epsilon = 1

def select_action(state):
    if random.random() < epsilon:
        return random.randint(0, 4)
    return model(torch.tensor(state).to(device).float().unsqueeze(0))[0].max(0)[1].view(1, 1).item()

Пришло время, наконец, обучить нашу модель

In [ ]:
last_four_frames = deque(maxlen=4)

for i in range(4):
    last_four_frames.append(frame)
actual_state = np.swapaxes(np.concatenate(last_four_frames, axis=2), 2, 0)

In [ ]:
max_step = 100001
for step in range(1, max_step):
    epsilon = 1 - 0.9 * step / max_step
    action = select_action(actual_state)
    state, reward, done, _ = env.step(action)
    last_four_frames.append(np.expand_dims(preprocesing(state), 2))

    if done:
        #We need to reset environment and frames deque
        memory.push((actual_state, action, reward, actual_state, done))
        frame = env.reset()
        for i in range(4):
            last_four_frames.append(np.expand_dims(preprocesing(frame), 2))
        done = False
        actual_state = np.swapaxes(np.concatenate(last_four_frames, axis=2), 2, 0)
    else:
        new_state = np.swapaxes(np.concatenate(last_four_frames, axis=2), 2, 0)
        memory.push((actual_state, action, reward, new_state, done))
        actual_state = new_state

    if step > 32:
        fit(list(zip(*memory.sample(32))))

    if step % 1000 == 0:
        target_model = copy.deepcopy(model)
        print("Progress", step // 1000, "%")